In [ ]:
import tensorflow as tf
import tensorflow.keras
from keras import layers
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.utils import shuffle, class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow_addons.losses import SigmoidFocalCrossEntropy
import pandas as pd
import os

In [ ]:
tf.__version__

In [ ]:
from sklearn.model_selection import train_test_split

#Load data csv with image paths and labels
train, test = train_test_split(data, test_size=0.4, random_state=1)
valid, test = train_test_split(test, test_size=.5, random_state=1)


In [ ]:
#central crop on 224 x 224
IMAGE_WIDTH, IMAGE_HEIGHT = (224, 224)
EPOCHS = 50
BATCH_SIZE = 32
learning = 0.0001
image_shape = (IMAGE_HEIGHT, IMAGE_WIDTH, 3)

In [ ]:
#zero mean to resize -1 to 1 
train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        
    )
valid_datagen = ImageDataGenerator(
        rescale=1./255
    )



In [ ]:
train_generator=train_datagen.flow_from_dataframe(
    dataframe=train, 
    directory=None,
    x_col="Path", y_col="Label", 
    class_mode="binary", 
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH), 
    batch_size=BATCH_SIZE,
    seed=16
)
valid_generator=valid_datagen.flow_from_dataframe(
    dataframe=valid, directory=None,
    x_col="Path", y_col="Label", 
    class_mode="binary", 
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH), 
    shuffle = True, 
    batch_size=BATCH_SIZE,
    seed=16
)
test_generator=valid_datagen.flow_from_dataframe(
    dataframe=test, 
    directory=None, 
    x_col="Path", y_col="Label", 
    class_mode="binary", 
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH), 
    shuffle = False, 
    batch_size=BATCH_SIZE,
    seed=16
)

In [ ]:
#create model
from keras.models import load_model
model = load_model("/home/jupyter-zaiman/COVID-19 Classification/ResNet50-UnderSampling/ResNet50_Undersampling.h5")
# model.layers[-1]._name='visualized_layer'
model.summary()

In [ ]:
model.compile(optimizer=Adam(lr=learning), loss='binary_crossentropy', metrics=['accuracy'])


save = ModelCheckpoint(
    '/home/jupyter-zaiman/COVID-19 Classification/ResNet50_covid/', 
    monitor='val_accuracy', 
    save_best_only=True,
    save_weights_only=True, 
    mode='max',
    verbose=1
)
stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=6)
scheduler = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.01,
    patience=3,
    verbose=1
    )
logger = CSVLogger('/home/jupyter-zaiman/COVID-19 Classification/ResNet50_covid/training.log')
history = model.fit(train_generator, 
    epochs=EPOCHS, 
    validation_data=valid_generator, 
    verbose=1, 
    callbacks=[save, scheduler, stop, logger,]
)



In [ ]:
model.summary()

In [ ]:
model.save("resnet_covid.hdf5")

In [ ]:
import seaborn as sns
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()
  
Y_pred = model.predict(test_generator, len(test_generator.filenames)) 
labels = (Y_pred >= 0.5).astype(np.int)
print('Confusion Matrix')
cm = confusion_matrix(test_generator.classes, labels)
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt="d");
target_names = ['negative', 'positive']
print('Classification Report')
print(classification_report(test_generator.classes, labels, target_names=target_names)) 

In [ ]:
def ppv(cm):
    ppv = cm[1, 1] / (cm[0, 1] + cm[1, 1])
    return ppv

def npv(cm):
    npv = cm[0, 0] / (cm[0, 0] + cm[1, 0])
    return npv
print(ppv(cm))
print(npv(cm))

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(test_generator.classes, Y_pred)

def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.title("ROC Curve")
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')

plot_roc_curve(fpr, tpr)
plt.show()
from sklearn.metrics import roc_auc_score
roc = roc_auc_score(test_generator.classes, Y_pred)
print("ROC AUC Score: " + roc.astype(str))

In [ ]:
from sklearn.metrics import precision_recall_curve, auc

precision, recall, thresholds = precision_recall_curve(test_generator.classes, Y_pred)
plt.title("PR Curve")
plt.plot(recall, precision)
plt.show()
auc_precision_recall = auc(recall, precision)
print("PR AUC: " + auc_precision_recall.astype(str))
